# Descriptive Analytics in Python

We will explore fundamental concepts of descriptive analytics through **customer segmentation**

We aim to use `pandas` to conduct **rule-based RFM (Recency, Frequency, Monetary Value)** clustering analysis to identify segments with distinct behaviors:

* **Recency** - days since last customer transaction. To define this with respect to the data's date coverage, assume that you are analyzing the data on Jan 1, 2022.
* **Frequency** - number of transactions in the last 12 months
* **Monetary Value** - total spend in the last 12 months



In [1]:
# Import libraries
import pandas as pd

In [2]:
# Mount GDrive's folders
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# This code imports a library "os" that allows file navigation
import os
# This code sets the home directory
# Find your folder and put the path here as a string
os.chdir('/content/drive/MyDrive/my_workspace')

## Prepare data

Read csv

In [4]:
df = pd.read_csv("Data/cc_clean.csv")
df.head()

,cc_num,gender,city,city_pop,job,dob,acct_num,acct_num2,trans_num,unix_time,category,amt,trans_datetime
0,6.760000e+11,M,Dasmarinas,659019,Chartered loss adjuster,12/12/1958,798000000000,798000000000,a72eaa86b043eed95b25bbb25b3153a1,1581314011,shopping_net,68.88,2020-02-10 13:53:31
1,3.520000e+15,M,Digos,169393,"Administrator, charities/voluntary organisations",31/08/1970,968000000000,968000000000,060d12f91c13871a13963041736a4702,1590902968,entertainment,50.06,2020-05-31 13:29:28
2,4.140000e+18,M,Calapan,133893,Financial controller,23/07/1953,628000000000,628000000000,18aafb6098ab0923886c0ac83592ef8d,1585461157,food_dining,105.44,2020-03-29 13:52:37
3,4.870000e+15,M,San Fernando,121812,NaN,18/07/1964,863000000000,863000000000,45bbe714e51ab8a375454d39a190b0cb,1613063704,food_dining,38.61,2021-02-12 01:15:04
4,4.720000e+15,M,Laoag,111125,Dance movement psychotherapist,11/01/1954,257000000000,257000000000,c20ee88b451f637bc6893b7460e9fee0,1601282159,gas_transport,82.69,2020-09-28 16:35:59


*Trick*: Pandas has its own methods for dates. We just need to convert it using `pd.to_datetime`.

In [6]:
df.dtypes

cc_num            float64
gender             object
city               object
city_pop            int64
job                object
dob                object
acct_num            int64
acct_num2           int64
trans_num          object
unix_time           int64
category           object
amt               float64
trans_datetime     object
dtype: object

In [7]:
df['trans_datetime'] = pd.to_datetime(df['trans_datetime'])
df.head()

,cc_num,gender,city,city_pop,job,dob,acct_num,acct_num2,trans_num,unix_time,category,amt,trans_datetime
0,6.760000e+11,M,Dasmarinas,659019,Chartered loss adjuster,12/12/1958,798000000000,798000000000,a72eaa86b043eed95b25bbb25b3153a1,1581314011,shopping_net,68.88,2020-02-10 13:53:31
1,3.520000e+15,M,Digos,169393,"Administrator, charities/voluntary organisations",31/08/1970,968000000000,968000000000,060d12f91c13871a13963041736a4702,1590902968,entertainment,50.06,2020-05-31 13:29:28
2,4.140000e+18,M,Calapan,133893,Financial controller,23/07/1953,628000000000,628000000000,18aafb6098ab0923886c0ac83592ef8d,1585461157,food_dining,105.44,2020-03-29 13:52:37
3,4.870000e+15,M,San Fernando,121812,NaN,18/07/1964,863000000000,863000000000,45bbe714e51ab8a375454d39a190b0cb,1613063704,food_dining,38.61,2021-02-12 01:15:04
4,4.720000e+15,M,Laoag,111125,Dance movement psychotherapist,11/01/1954,257000000000,257000000000,c20ee88b451f637bc6893b7460e9fee0,1601282159,gas_transport,82.69,2020-09-28 16:35:59


Check transaction date coverage

In [8]:
df['trans_datetime'].min()

Timestamp('2020-01-01 08:06:15')

In [9]:
df['trans_datetime'].max()

Timestamp('2021-12-07 07:50:42')

For recency analysis, we need to know how much days have passed since the transaction, assuming current date is Jan 1, 2022.

>Q: Head to [ChatGPT](https://chat.openai.com/) and create a prompt to instruct it to write code that can produce a new dataframe column containing elapsed days since date in column `trans_datetime`, assuming current date is Jan 1, 2022

In [14]:
# Define the current date
current_date = pd.to_datetime('2022-01-01')

# Calculate the elapsed days
df['elapsed_days'] = (current_date - df['trans_datetime']).dt.days

In [15]:
df.head()

,cc_num,gender,city,city_pop,job,dob,acct_num,acct_num2,trans_num,unix_time,category,amt,trans_datetime,elapsed_days
0,6.760000e+11,M,Dasmarinas,659019,Chartered loss adjuster,12/12/1958,798000000000,798000000000,a72eaa86b043eed95b25bbb25b3153a1,1581314011,shopping_net,68.88,2020-02-10 13:53:31,690
1,3.520000e+15,M,Digos,169393,"Administrator, charities/voluntary organisations",31/08/1970,968000000000,968000000000,060d12f91c13871a13963041736a4702,1590902968,entertainment,50.06,2020-05-31 13:29:28,579
2,4.140000e+18,M,Calapan,133893,Financial controller,23/07/1953,628000000000,628000000000,18aafb6098ab0923886c0ac83592ef8d,1585461157,food_dining,105.44,2020-03-29 13:52:37,642
3,4.870000e+15,M,San Fernando,121812,NaN,18/07/1964,863000000000,863000000000,45bbe714e51ab8a375454d39a190b0cb,1613063704,food_dining,38.61,2021-02-12 01:15:04,322
4,4.720000e+15,M,Laoag,111125,Dance movement psychotherapist,11/01/1954,257000000000,257000000000,c20ee88b451f637bc6893b7460e9fee0,1601282159,gas_transport,82.69,2020-09-28 16:35:59,459


--------------------------------------
## Perform aggregations

We can use pandas aggregation functions to understand customer transaction behaviors

### Aggregate using `.value_counts()`
This gets the total number of entries per unique item in one column

> Q: Which customers (based on column `acct_num`) have the most transactions?



In [16]:
df['acct_num'].value_counts()

acct_num
902000000000    4231
845000000000    3531
233000000000    2834
581000000000    2157
439000000000    2125
                ... 
647000000000       8
224000000000       7
644000000000       7
229000000000       7
138000000000       7
Name: count, Length: 89, dtype: int64

> Q: Which cities have the most transactions?



In [17]:
df['city'].value_counts()

city
San Fernando          6371
Calapan               4959
Masbate               4950
Cadiz                 4940
Dasmarinas            4250
Malaybalay            3541
Pagadian              3540
Olongapo              3524
Dumaguete City        2823
City of Isabela       2136
Muntinlupa City       2132
Mati                  2131
Butuan                2129
Malabon               2129
Tagbilaran City       2128
Sorsogon              2126
Angeles City          2121
City of Calamba       2121
Meycauayan            2120
Naga City             2118
Santa Rosa            2118
Binan                 2117
Batangas              2117
Digos                 2115
Makati City           2113
Laoag                 2113
Tacloban              2110
Maasin                1421
General Santos        1421
Palayan City          1419
Puerto Princesa       1418
Navotas               1418
Vigan                 1417
Balanga               1417
Cabanatuan City       1415
Valenzuela            1412
City of Paranaque     1

### Aggregate using `.groupby() `

Creates subsets of the dataframe by grouping based on a column, and performs aggregations on each group. The whole process involves:

- **Splitting** data into groups based on some features.
- **Applying** a function to each group independently.
- **Combining** the result into data structure.

![pandas `groupby` explained](https://learnsql.com/blog/group-by-in-sql-explained/GROUP_BY-sum.jpg)

The basic syntax of a groupby is:

```python
# single column key, single column value, single aggregation
df.groupby("<key_column_name1>")["value_column_name1"].<aggregation function>()

# multiple column key, multiple column value, single aggregation
df.groupby(["<key_column_name1>","<key_column_name2>"])[["value_column_name1","value_column_name2"]].<aggregation function>()

# multiple column key, multiple column value, multiple aggregation
df.groupby(["<key_column_name1>","<key_column_name2>"])[["value_column_name1","value_column_name2"]].agg(['<agg_func1>,<agg_func2>])

```

The following are the default aggregation functions you can use in `groupby`

* `sum`: Computes the sum of values.
* `mean`: Computes the mean of values.
* `median`: Computes the median of values.
* `min`: Computes the minimum of values.
* `max`: Computes the maximum of values.
* `size`: Counts the number of values, including nulls.
* `count`: Counts the number of non-null values.
* `std`: Computes the standard deviation of values.
* `var`: Computes the variance of values.
* `first`: Returns the first value in each group.
* `last`: Returns the last value in each group.
* `nunique`: Returns count of unique elements in each group

In [18]:
df.head()

,cc_num,gender,city,city_pop,job,dob,acct_num,acct_num2,trans_num,unix_time,category,amt,trans_datetime,elapsed_days
0,6.760000e+11,M,Dasmarinas,659019,Chartered loss adjuster,12/12/1958,798000000000,798000000000,a72eaa86b043eed95b25bbb25b3153a1,1581314011,shopping_net,68.88,2020-02-10 13:53:31,690
1,3.520000e+15,M,Digos,169393,"Administrator, charities/voluntary organisations",31/08/1970,968000000000,968000000000,060d12f91c13871a13963041736a4702,1590902968,entertainment,50.06,2020-05-31 13:29:28,579
2,4.140000e+18,M,Calapan,133893,Financial controller,23/07/1953,628000000000,628000000000,18aafb6098ab0923886c0ac83592ef8d,1585461157,food_dining,105.44,2020-03-29 13:52:37,642
3,4.870000e+15,M,San Fernando,121812,NaN,18/07/1964,863000000000,863000000000,45bbe714e51ab8a375454d39a190b0cb,1613063704,food_dining,38.61,2021-02-12 01:15:04,322
4,4.720000e+15,M,Laoag,111125,Dance movement psychotherapist,11/01/1954,257000000000,257000000000,c20ee88b451f637bc6893b7460e9fee0,1601282159,gas_transport,82.69,2020-09-28 16:35:59,459


> Q: On average, how much is spent on each transaction per category?



In [19]:
df.groupby('category')['amt'].mean()

category
entertainment      51.427389
food_dining        80.358671
gas_transport      69.207172
grocery_net        88.654834
grocery_pos        94.601127
health_fitness     40.465937
home               60.308439
kids_pets          79.744617
misc_net           83.135981
misc_pos           61.528609
personal_care      44.829610
shopping_net      120.214624
shopping_pos       60.077685
travel             77.971262
Name: amt, dtype: float64

>Q: Can you arrange the result from greatest to least?

In [20]:
df.groupby('category')['amt'].mean().sort_values(ascending=False)

category
shopping_net      120.214624
grocery_pos        94.601127
grocery_net        88.654834
misc_net           83.135981
food_dining        80.358671
kids_pets          79.744617
travel             77.971262
gas_transport      69.207172
misc_pos           61.528609
home               60.308439
shopping_pos       60.077685
entertainment      51.427389
personal_care      44.829610
health_fitness     40.465937
Name: amt, dtype: float64

> Q: What is the maximum amount spent for all transaction per category? Arrange from greatest to least.



In [21]:
df.groupby('category')['amt'].max().sort_values(ascending=False)

category
travel            18039.81
shopping_pos       3371.31
shopping_net       2435.60
misc_pos           1958.89
misc_net           1865.62
entertainment       962.12
health_fitness      517.29
personal_care       477.01
food_dining         470.51
home                450.45
kids_pets           423.35
grocery_pos         355.26
grocery_net         168.52
gas_transport       134.63
Name: amt, dtype: float64

> Q: What is the total amount spent in each city? Arrange from greatest to least.



In [22]:
df.groupby('city')['amt'].sum().sort_values(ascending=False)

city
San Fernando          427733.38
Masbate               356236.82
Cadiz                 340712.29
Calapan               334181.28
Dasmarinas            299195.32
Pagadian              265570.97
Olongapo              246109.34
Malaybalay            242356.81
Dumaguete City        196750.01
Binan                 162070.92
Sorsogon              161800.49
City of Calamba       158236.83
City of Isabela       152041.12
Meycauayan            149704.41
Batangas              149699.10
Tagbilaran City       148118.48
Mati                  145875.56
Santa Rosa            142610.18
Tacloban              142467.79
Angeles City          142158.97
Laoag                 141812.74
Butuan                141687.94
Naga City             140190.98
Muntinlupa City       139305.39
Digos                 137061.06
Makati City           135864.37
Malabon               135372.15
Caloocan City         114183.11
Navotas               102780.45
Palayan City          102557.71
General Santos        101137.31
Bal

>Q: How many unique customers are present per gender?

In [23]:
df.groupby('gender')['acct_num'].nunique()

gender
F     6
M    84
Name: acct_num, dtype: int64

> Q: What is the total and average amount spent per category?



In [24]:
df.groupby('category')['amt'].agg(['sum','mean'])

,sum,mean
category,,
entertainment,366163.01,51.427389
food_dining,587582.60,80.358671
gas_transport,648194.37,69.207172
grocery_net,138833.47,88.654834
grocery_pos,884993.54,94.601127
health_fitness,203988.79,40.465937
home,546997.54,60.308439
kids_pets,731337.88,79.744617
misc_net,258137.22,83.135981


> Q: What is the average amount spent for transactions per category, in each city?



In [26]:
df.groupby(['city','category'])['amt'].mean().sort_values(ascending=False)

city               category    
Mandaluyong City   shopping_net    1031.025000
Talisay            shopping_net    1027.317500
Legazpi City       shopping_net    1017.095000
                   misc_net        1016.320000
Iligan             shopping_net    1012.290000
                                      ...     
                   misc_pos           6.180000
City of Paranaque  travel             6.016905
Marikina City      travel             5.910000
Cabanatuan City    travel             5.874103
Dipolog            misc_pos           3.670000
Name: amt, Length: 741, dtype: float64

## RFM Example
Determine some spending characteristics of each customer in the past 12 months


First, create a per-customer table using transactions done in 2021

In [27]:
# Filter data to 2021
data_df = df[df['trans_datetime'].dt.year==2021]
data_df

,cc_num,gender,city,city_pop,job,dob,acct_num,acct_num2,trans_num,unix_time,category,amt,trans_datetime,elapsed_days
3,4.870000e+15,M,San Fernando,121812,NaN,18/07/1964,863000000000,863000000000,45bbe714e51ab8a375454d39a190b0cb,1613063704,food_dining,38.61,2021-02-12 01:15:04,322
7,1.800000e+14,M,Balanga,96061,Manufacturing systems engineer,14/05/1950,233000000000,233000000000,c371800cdd75b7373b0a883e9ea38450,1633018557,kids_pets,47.87,2021-10-01 00:15:57,91
9,4.010000e+12,M,Malaybalay,174625,Tourist information centre manager,11/01/1966,439000000000,439000000000,2ecae6c920f6ae3e3cf3dd3abd225fda,1621672603,NaN,114.37,2021-05-22 16:36:43,223
10,3.470000e+14,M,Antipolo,776386,Seismic interpreter,19/05/1959,564000000000,564000000000,44fec72badccf385eec1f6bf27ccdf65,1612980825,personal_care,58.04,2021-02-11 02:13:45,323
14,1.800000e+14,M,Marikina City,450741,NaN,12/08/1955,34614744133,34614744133,3a86393194aa83069765af258572b4cb,1615174156,shopping_pos,127.83,2021-03-08 11:29:16,298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99993,6.550000e+15,M,Malaybalay,174625,English as a foreign language teacher,31/12/1948,581000000000,581000000000,40b006a3e501f90c47ae5730e7e17316,1617759699,grocery_pos,60.76,2021-04-07 09:41:39,268
99995,2.470000e+15,M,San Fernando,306659,"Surveyor, quantity",01/11/1935,581000000000,581000000000,d44f411eabd406a76a60546e723a98fd,1628185569,kids_pets,98.23,2021-08-06 01:46:09,147
99996,3.520000e+15,M,Masbate,95389,Wellsite geologist,20/11/1967,531000000000,531000000000,7e767a74cae901c13f1a9d1d37aa63d4,1621481285,grocery_pos,78.79,2021-05-20 11:28:05,225
99997,4.620000e+15,M,San Fernando,121812,Personnel officer,20/11/1934,555000000000,555000000000,6ced184c93e66028e8d235ad3060de90,1625341374,personal_care,31.37,2021-07-04 03:42:54,180


### Recency
>Q: How many days have passed since the last transaction of each customer?

In [28]:
# Get days since last transaction of each customer first
# last transaction min of elapsed time
# groupby + trick to convert back to dataframe
rec_df = data_df.groupby(['acct_num'])['elapsed_days'].min().to_frame(name='recency').reset_index()
rec_df

,acct_num,recency
0,3843260959,25
1,20386303916,111
2,24058295524,240
3,34614744133,25
4,38399733986,25
...,...,...
78,968000000000,25
79,969000000000,25
80,971000000000,25
81,989000000000,25


In [ ]:
# view quantiles
rec_df[rec_df['recency']>25]['recency'].describe()

In [29]:
# view quantiles
rec_df['recency'].describe()

count     83.000000
mean      58.879518
std       78.496139
min       24.000000
25%       24.000000
50%       25.000000
75%       25.000000
max      339.000000
Name: recency, dtype: float64

Using this, create a system to assign a recency score to the `recency` column.

**More recent transaction => Higher score**

* 1 - low score
* 2 - medium score
* 3 - high score

In [33]:
def assign_score_recency(x):
  if x<=30: # last transaction is within last month
    return 3
  elif 30<x<90: # within last quarter
    return 2
  else: #older than last quarter
    return 1

In [31]:
rec_df['recency_score'] = rec_df['recency'].map(assign_score_recency)
rec_df

,acct_num,recency,recency_score
0,3843260959,25,3
1,20386303916,111,1
2,24058295524,240,1
3,34614744133,25,3
4,38399733986,25,3
...,...,...,...
78,968000000000,25,3
79,969000000000,25,3
80,971000000000,25,3
81,989000000000,25,3


In [34]:
rec_df['recency_score'].value_counts()

recency_score
3    66
1    16
2     1
Name: count, dtype: int64

### Frequency
>Q: On average, how frequent did each customer  transact in the past year?

In [43]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46867 entries, 3 to 99998
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   cc_num          46867 non-null  float64       
 1   gender          46867 non-null  object        
 2   city            46867 non-null  object        
 3   city_pop        46867 non-null  int64         
 4   job             43274 non-null  object        
 5   dob             46867 non-null  object        
 6   acct_num        46867 non-null  int64         
 7   acct_num2       46867 non-null  int64         
 8   trans_num       46867 non-null  object        
 9   unix_time       46867 non-null  int64         
 10  category        43322 non-null  object        
 11  amt             46867 non-null  float64       
 12  trans_datetime  46867 non-null  datetime64[ns]
 13  elapsed_days    46867 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(5), object(6)
memor

In [38]:
# Get transaction frequency of each customer first
# groupby + trick to convert back to dataframe
freq_df = data_df.groupby(['acct_num']).size().to_frame(name='frequency').reset_index()
freq_df

,acct_num,frequency
0,3843260959,1022
1,20386303916,12
2,24058295524,11
3,34614744133,344
4,38399733986,334
...,...,...
78,968000000000,1053
79,969000000000,325
80,971000000000,984
81,989000000000,324


Using pandas method `pd.qcut` (quantile cut), classify and score the customers based on their frequency

**More frequent transactions => Higher score**

* 1- low score
* 2- medium score
* 3- high score

In [39]:
freq_df['frequency'].describe()

count      83.000000
mean      564.662651
std       423.617415
min         5.000000
25%       323.500000
50%       636.000000
75%       974.000000
max      1995.000000
Name: frequency, dtype: float64

In [40]:
freq_df['frequency_score'] = pd.qcut(freq_df['frequency'], q=3, labels=[1, 2, 3]).astype(int)
freq_df

,acct_num,frequency,frequency_score
0,3843260959,1022,3
1,20386303916,12,1
2,24058295524,11,1
3,34614744133,344,2
4,38399733986,334,2
...,...,...,...
78,968000000000,1053,3
79,969000000000,325,1
80,971000000000,984,3
81,989000000000,324,1


In [41]:
freq_df['frequency_score'].value_counts()

frequency_score
1    29
3    28
2    26
Name: count, dtype: int64

### Monetary value
>Q: On average, how much is the total spending of each customerin the past year?

In [45]:
# Get total spending of each customer first
# groupby + trick to convert back to dataframe
mv_df = data_df.groupby(['acct_num'])['amt'].sum().to_frame(name='total_amt').reset_index()
mv_df

,acct_num,total_amt
0,3843260959,69812.15
1,20386303916,8803.87
2,24058295524,6236.55
3,34614744133,21803.56
4,38399733986,24769.02
...,...,...
78,968000000000,62850.52
79,969000000000,22380.54
80,971000000000,66340.55
81,989000000000,20749.72


Using pandas method `pd.qcut` (quantile cut), classify and score the customers based on their monetary value

**Larger total transaction amts => Higher score**

* 1- low score
* 2- medium score
* 3- high score

In [46]:
mv_df['monetary_score'] = pd.qcut(mv_df['total_amt'], q=3, labels=[1, 2, 3]).astype(int)
mv_df

,acct_num,total_amt,monetary_score
0,3843260959,69812.15,3
1,20386303916,8803.87,1
2,24058295524,6236.55,1
3,34614744133,21803.56,1
4,38399733986,24769.02,2
...,...,...,...
78,968000000000,62850.52,3
79,969000000000,22380.54,1
80,971000000000,66340.55,3
81,989000000000,20749.72,1


In [47]:
mv_df['monetary_score'].value_counts()

monetary_score
3    28
1    28
2    27
Name: count, dtype: int64

In [48]:
# Q: On average, how much do each customer spend per category/score?
mv_df.groupby('monetary_score')['total_amt'].mean()

monetary_score
1    11410.112500
2    36466.882593
3    72843.987143
Name: total_amt, dtype: float64

### Total RFM score
To compute the total score, we need to


1.   Merge the recency, frequency, and monetary value tables together
2.   Add the scores to a column `rfm_score`



Pandas has a method called `merge` that quickly accomplishes table merging tasks. There are 4 kinds of merges, specified by the `how` argument

![](https://miro.medium.com/v2/resize:fit:1400/1*rMW6TQfoBwfOZrFhTdxUVg.png)

In [49]:
# First, merge rec_df and freq_df using pd.merge
rfm_df = pd.merge(rec_df, freq_df, on='acct_num', how='inner')
rfm_df.head()

,acct_num,recency,recency_score,frequency,frequency_score
0,3843260959,25,3,1022,3
1,20386303916,111,1,12,1
2,24058295524,240,1,11,1
3,34614744133,25,3,344,2
4,38399733986,25,3,334,2


In [50]:
# check lengths to see if we did merge correctly
print(len(rec_df))
print(len(freq_df))
print(len(rfm_df))

83
83
83


In [51]:
# Then merge `mv_df` to `rfm_df`
rfm_df = pd.merge(rfm_df, mv_df, on='acct_num', how='inner')
rfm_df.head()

,acct_num,recency,recency_score,frequency,frequency_score,total_amt,monetary_score
0,3843260959,25,3,1022,3,69812.15,3
1,20386303916,111,1,12,1,8803.87,1
2,24058295524,240,1,11,1,6236.55,1
3,34614744133,25,3,344,2,21803.56,1
4,38399733986,25,3,334,2,24769.02,2


In [52]:
# check lengths to see if we did merge correctly
print(len(mv_df))
print(len(rfm_df))

83
83


Add up the scores and store in column `rfm_score`

In [54]:
rfm_df['rfm_score'] = rfm_df['recency_score'] + rfm_df['frequency_score'] +rfm_df['monetary_score']

In [55]:
rfm_df.head()

,acct_num,recency,recency_score,frequency,frequency_score,total_amt,monetary_score,rfm_score
0,3843260959,25,3,1022,3,69812.15,3,9
1,20386303916,111,1,12,1,8803.87,1,3
2,24058295524,240,1,11,1,6236.55,1,3
3,34614744133,25,3,344,2,21803.56,1,6
4,38399733986,25,3,334,2,24769.02,2,7


In [56]:
rfm_df['rfm_score'].value_counts()

rfm_score
9    27
7    22
3    16
5     8
6     7
8     2
4     1
Name: count, dtype: int64

Create function to assign RFM levels

In [57]:
def assign_rfm_level(x):
    if x >= 7:
        return 'Top'
    elif 4<=x<7:
        return 'Middle'
    else:
        return 'Low'


In [58]:
rfm_df['rfm_level'] = rfm_df['rfm_score'].map(assign_rfm_level)
rfm_df.head()

,acct_num,recency,recency_score,frequency,frequency_score,total_amt,monetary_score,rfm_score,rfm_level
0,3843260959,25,3,1022,3,69812.15,3,9,Top
1,20386303916,111,1,12,1,8803.87,1,3,Low
2,24058295524,240,1,11,1,6236.55,1,3,Low
3,34614744133,25,3,344,2,21803.56,1,6,Middle
4,38399733986,25,3,334,2,24769.02,2,7,Top


Now you have an RFM score to each of your customers, which you can use for targeted strategies